<a href="https://colab.research.google.com/github/Fhooijmans/MasterThesis/blob/main/Preprocessing_%26_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import dataset
import pandas as pd
import numpy as np
df = pd.read_csv('release_10_23_2020.csv')
print(df)


######## Pre-Processing
# Create df with sequence column
# copy orignal dataset to new dataframe
df_copy = df.copy()

#replace nan values with pageview in product_action column
df_copy["product_action"].fillna("pageview", inplace=True)
df_copy["product_action"].values


#add column to df with event_symbols
event_dict = {"pageview" : 1, "detail" : 2, "add" : 3, "remove" : 4, "click" : 5, "purchase" : 6}
    #create list and fill with symbols via for-loop
symbol_events = []
for action in df_copy["product_action"]:
    symbol_events.append(event_dict[action]) 
    #add list to dataframe
df_copy['symbol_event'] = symbol_events #list with symbols corresponding to product_action column added to df

#get unique session id list from original dataset
list_all_sessions = df_copy['session_id_hash'].copy() #series of all id occurences in dataset, same nr of rows as symbol_event list
list_unique_sessions = list_all_sessions.unique() #list with unique ids

#df with unique ids, to later append sequences to
df_sequences = pd.DataFrame(list_unique_sessions.copy(), columns=['session_id'])
df_sequences['sequence_events'] = "" #added empty sequence column for appending

count_ids = 0 #index of unique id
count_events = 0 #index to know at which event we are in the total symbol list, to not have to iterate over whole list for every session id
for id_unique in list_unique_sessions:
    list_events = [] # for each unique session id, an empty list is created to append the individual events to of that session
    for id_all in list_all_sessions[count_events::]: 
# =============================================================================
# Start of inner loop is based on event_counter, to prevent iterating over events that already have been processed
# Count_events registers which event rows have been processed. Loop continues at session id row that made previous loop 'Break'.
# This is less computational intensive because now this inner loop only iterates over the number of rows that match the sess_id + 1.
# Using 'Continue' instead of 'Break' would make the inner loop also iterate over all remaining id_record rows (i.e. events) 
# after the final row that matched the currently checked sess_id, including ids that don't match the current checked sess_id, 
# which is is unnecessary compsumption of computational power.
# =============================================================================
        if id_unique == id_all:
            list_events.append(symbol_events[count_events]) #symbol _event is separate list with all events in order
        else:
            break #When a row w. different id than sess_id is reached, loop breaks
        count_events += 1 # counter + 1 for every event added to list_events
    df_sequences.loc[count_ids, 'sequence_events'] = list_events #assigns sequence to corresponding session
    count_ids += 1
    
#df1 = df.groupby('session_id_hash')['symbol'].apply(list).reset_index(name='sequence') 
## group by session and make list of symbols who belong to session

#Export new df to CSV
df_sequences.to_csv(r'all_sequences.csv', index = False)

# Import df created above
df_add_sequences = pd.read_csv('all_sequences.csv', converters={'sequence_events': eval})
#pd.read_csv('all_sequences.csv', converters={'sequence_events': eval}) #keeps sequence column as list

#convert sequence column from string to list -- sequences are strings when imported instead of lists
df_add_sequences['sequence_events'] = df_add_sequences['sequence_events'].str.replace('[', "")
df_add_sequences['sequence_events'] = df_add_sequences['sequence_events'].str.replace(']', "")
#drop rows without 'add' event in sequence (i.e. '3')
df_add_sequences = df_add_sequences[df_add_sequences['sequence_events'].str.contains('3')]
df_add_sequences['sequence_events'] = df_add_sequences['sequence_events'].str.split(', ') # converts strings to lists
df_add_sequences = df_add_sequences.reset_index(drop=True) #reset row index after dropping 'no add' rows

### Trim purchase sequences to remove indication of purchase, and add P/A label column, P: Purchase, A: Abandonment (i.e. make purchase)
    # add P/A label first
df_add_sequences['target_label'] = ""
    # Enumerate can be used to count row index when starting at first row and iterating over all rows
for index, seq in enumerate(df_add_sequences['sequence_events']):
    if '6' in seq:
        df_add_sequences.loc[index, 'target_label'] = 'P'
    else:
        df_add_sequences.loc[index, 'target_label'] = 'A'

    #Export df to CSV
df_add_sequences.to_csv(r'add_sequences.csv', index = False)
df_add_seq_trim = pd.read_csv('add_sequences.csv', converters={'sequence_events': eval}) # import updated new CSV
    #trim sequences by removing purchase symbol (6), and subsequent symbols
for seq in df_add_seq_trim['sequence_events']:
    if '6' in seq:
        index = seq.index('6')
        length = len(seq)
        seq[index:length] = ""


### Add column w. sequence length, to give insight in lengths of all 'add' sequences till Purchase (P) OR Abandonment (A)
    # empty column to fill w. seq lengths
df_add_seq_trim['sequence_length'] = ""
    # fill length column
for index, seq in enumerate(df_add_seq_trim['sequence_events']):
    seq_length = len(seq)
    df_add_seq_trim.loc[index, 'sequence_length'] = seq_length
    # Export df to CSV
df_add_seq_trim.to_csv(r'length_sequences.csv', index = False) 

df_length = pd.read_csv('length_sequences.csv', converters={'sequence_events': eval}) # import updated new CSV

# Removing too short/low sequences, this includes P seqs that were longer before trimming as done above
    #copy of orignal df
df_length = df_add_seq_trim.copy()
    #drop rows with sequences of length <5
df_length = df_length.drop(df_length[df_length.sequence_length < 5].index)
    #drop rows with sequences of length >155
df_length = df_length.drop(df_length[df_length.sequence_length > 155].index)

df_length.to_csv(r'length_trim_sequences.csv', index = False)

df_length = pd.read_csv('length_trim_sequences.csv', converters={'sequence_events': eval})

######## EDA/Plots
                                                #####Sequence length plots
import matplotlib.pyplot as plt
    #Length distribution after trim (4 < L < 156)
df_abandon = df_length.drop(df_length[df_length.target_label == 'P'].index)
df_purchase = df_length.drop(df_length[df_length.target_label == 'A'].index)
fig = plt.figure(figsize=(8,4), dpi=150)
plt.hist(df_abandon['sequence_length'], bins=75, histtype='stepfilled', density=True, alpha=0.5, log=True, color='red', label='A')
plt.hist(df_purchase['sequence_length'], bins=75, histtype='stepfilled', density=True, alpha=0.5, log=True, color='limegreen', label='P')
plt.ylim(0.0001, 1)
plt.title('Length distribution add-to-cart sequences')
plt.xlabel('Trajectory length L')
plt.ylabel('Probability')
plt.legend()
plt.grid(b=True)
plt.show()
fig.savefig('/Length_add.png', format='png')


                                                ####dist. of events plot
                                                
df_abandon['pageview_count'] = [seq.count('1') for seq in df_abandon['sequence_events']]
df_purchase['pageview_count'] = [seq.count('1') for seq in df_purchase['sequence_events']]

df_abandon['detail_count'] = [seq.count('2') for seq in df_abandon['sequence_events']]
df_purchase['detail_count'] = [seq.count('2') for seq in df_purchase['sequence_events']]

df_abandon['add_count'] = [seq.count('3') for seq in df_abandon['sequence_events']]
df_purchase['add_count'] = [seq.count('3') for seq in df_purchase['sequence_events']]

df_abandon['remove_count'] = [seq.count('4') for seq in df_abandon['sequence_events']]
df_purchase['remove_count'] = [seq.count('4') for seq in df_purchase['sequence_events']]

df_abandon['click_count'] = [seq.count('5') for seq in df_abandon['sequence_events']]
df_purchase['click_count'] = [seq.count('5') for seq in df_purchase['sequence_events']]


r1 = ['Abandon']
r2 = ['Purchase']
width = 0.8
# to percentage_abandon
totals_abandon = [i+j+k+l+m for i,j,k,l,m in zip( [sum(df_abandon['pageview_count'])], [sum(df_abandon['detail_count'])], [sum(df_abandon['add_count'])], [sum(df_abandon['remove_count'])], [sum(df_abandon['click_count'])]) ]
pageview_a = [i/j * 100 for i,j in zip([sum(df_abandon['pageview_count'])], totals_abandon)]
detail_a = [i/j * 100 for i,j in zip([sum(df_abandon['detail_count'])], totals_abandon)]
add_a = [i/j * 100 for i,j in zip([sum(df_abandon['add_count'])], totals_abandon)]
remove_a = [i/j * 100 for i,j in zip([sum(df_abandon['remove_count'])], totals_abandon)]
click_a = [i/j * 100 for i,j in zip([sum(df_abandon['click_count'])], totals_abandon)]

# to percentage purchase
totals_add = [i+j+k+l+m for i,j,k,l,m in zip( [sum(df_purchase['pageview_count'])], [sum(df_purchase['detail_count'])], [sum(df_purchase['add_count'])], [sum(df_purchase['remove_count'])], [sum(df_purchase['click_count'])]) ]
pageview_add = [i/j * 100 for i,j in zip([sum(df_purchase['pageview_count'])], totals_add)]
detail_add = [i/j * 100 for i,j in zip([sum(df_purchase['detail_count'])], totals_add)]
add_add = [i/j * 100 for i,j in zip([sum(df_purchase['add_count'])], totals_add)]
remove_add = [i/j * 100 for i,j in zip([sum(df_purchase['remove_count'])], totals_add)]
click_add = [i/j * 100 for i,j in zip([sum(df_purchase['click_count'])], totals_add)]


        #plot_abandon
#pageview
plt.bar(r1, pageview_a, color='#b5ffb9', edgecolor='white', label='pageview', width=width)
#detail
plt.bar(r1, detail_a, bottom=pageview_a, color='#f9bc86', edgecolor='white', label='detail', width=width)
#add
plt.bar(r1, add_a, bottom=[i+j for i,j in zip(pageview_a, detail_a)], color='#a3acff', edgecolor='white', label='add', width=width)    
#remove
plt.bar(r1, remove_a, bottom=[i+j+k for i,j,k in zip(pageview_a, detail_a, add_a)], color='#ffe873', edgecolor='white', label='remove', width=width)
#click
plt.bar(r1, click_a, bottom=[i+j+k+l for i,j,k,l in zip(pageview_a, detail_a, add_a, remove_a)], color='#ffc5bf', edgecolor='white', label='click', width=width)

                          
        #plot_purchase
plt.bar(r2, pageview_add, color='#b5ffb9', edgecolor='white', width=width)
#detail
plt.bar(r2, detail_add, bottom=pageview_add, color='#f9bc86', edgecolor='white', width=width)
#add
plt.bar(r2, add_add, bottom=[i+j for i,j in zip(pageview_add, detail_add)], color='#a3acff', edgecolor='white', width=width)    
#remove
plt.bar(r2, remove_add, bottom=[i+j+k for i,j,k in zip(pageview_add, detail_add, add_add)], color='#ffe873', edgecolor='white', width=width)
#click
plt.bar(r2, click_add, bottom=[i+j+k+l for i,j,k,l in zip(pageview_add, detail_add, add_add, remove_add)], color='#ffc5bf', edgecolor='white', width=width)                           

plt.title('Event distribution')                            
plt.xlabel('Label')
plt.ylabel('% of total events') 
plt.grid(b=True)      
plt.legend(loc='lower center')    
plt.show()

                                                    #####mosaic plot
#df_mosaic.to_csv(r'df_plot.csv', index = False)
df_mosaic = pd.read_csv('df_plot.csv', converters={'sequence_events': eval})
df_mosaic['remove_presence'] = ['present' if value != 0 else 'not present' for value in df_mosaic['remove_count']]

from statsmodels.graphics.mosaicplot import mosaic

props={}
props[('A','present')]={'facecolor':'xkcd:peach', 'edgecolor':'white'}
props[('P','present')]={'facecolor':'xkcd:light blue', 'edgecolor':'white'}
props[('A','not present')]={'facecolor':'xkcd:peach','edgecolor':'white'}
props[('P','not present')]={'facecolor':'xkcd:light blue','edgecolor':'white'}

labelizer = lambda k: {('A','present'):7645,('P','present'):3188,('A','not present'):22436,('P','not present'):3780}[k]
mosaic(df_mosaic,['target_label','remove_presence'], properties=props, labelizer=labelizer, title='Label comparison by "remove"-event presence')

plt.show()